In [7]:
import chit

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
chat = chit.Chat.clone("foo2.json")

In [9]:
chat.log("tree")

[S_]dfac3dae──[U*]203da29b──[A_]dab57316── (master)
            │             ├─[A_]2ad8645b── (retry_response)
            │             └─ (new_branch*)
            └─[U_]ecf21cad──[A_]af5de4e2──[U_]1aeaedd1──[A_]afacbdeb──[U_]f05b7bd4──[A_]8ee7390f── (edit_prompt)


In [10]:
import ast
import operator


def math_eval(expr: str) -> str | float:
    """
    Evaluate a simple mathematical expression. The supported operators are * (multiply), - (subtract),
    + (add), / (divide), and ** (power). If your expression was unable to be evaluated,
    the output will instead be "Invalid expression".

    Parameters
    ----------
    expr : str
        the expression to evaluate

    Returns
    -------
    float
        the result of the evaluation, or str "Invalid expression" if the expression is invalid
    """
    operators = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.USub: operator.neg,
        ast.Pow: operator.pow
    }
    def eval_node(node):
        if isinstance(node, ast.BinOp):
            if type(node.op) in operators:
                left = eval_node(node.left)
                right = eval_node(node.right)
                return operators[type(node.op)](left, right)
            else:
                raise ValueError(f"Unsupported operation {node.op}")
        elif isinstance(node, ast.Constant):
            return node.value
        else:
            raise ValueError(f"Unsupported type {type(node)}")

    try:
        # Parse the expression into an AST
        node = ast.parse(expr, mode='eval').body
        return str(eval_node(node))
    except Exception as e:
        return f"ERROR: {type(e)}"

In [11]:
chat.tools = [math_eval]
chat._recalc_tools()

In [12]:
chat.checkout(branch_name="edit_prompt")
chat.commit("Now I have given you the ability to evaluate simple mathematical expressions. Try it out!")
chat.commit()

Yes, I do have access to a math evaluation tool! Let me demonstrate it with a few examples:

1. Let's try a simple addition:
1 tool calls pending; use .commit() to call one-by-one


In [13]:
chat.commit("Think of any random numbers and add them using the math_eval tool I gave you.")

In [14]:
chat.commit()

None


In [15]:
chat.commit("Huh, see if you have access to math_eval now? Just add 547 and 623 and get back to me.")

In [17]:
chat.commit()

None


In [21]:
chat.commit("Hello? say something")
chat.commit()

None


In [22]:
chat.commit("Hello?!")
chat.commit(enable_tools=False)

In [23]:
chat.current_message

Message(id='1e2554d3', message={'role': 'assistant', 'content': ''}, children={'edit_prompt': None}, parent_id='bf46a00c', home_branch='edit_prompt', tool_calls=None)